<a href="https://colab.research.google.com/github/Rafavermar/PROJECTS_Python/blob/main/PROJECT%207%20OpenAI_Whisper_VIDEOtoText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🗣 **Whisper** - _**Forked from DotCSV's notebook with enhancements**_

🚨 Have **Hardware Acceleration** enabled with GPU in `"Runtime Environment" > "Change Runtime Environment Type"`

In [ ]:
#@title ⚙️ Run this cell to install the libraries.
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install ffmpeg-python
!pip install translate
!pip install youtube_transcript_api
!pip install summa
!pip install youtube-transcript-api

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-gsmgz4bv
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-gsmgz4bv
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798075 sha256=3d38f9c4d54725765e42fea0b6c9b672b2ab0ea76deebc60bc55358a9444f8f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-cxawhs2y/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Looking in indexes: https://pypi.org/simple, https://us-p

##1. TRANSCRIBE FROM A HOSTED VIDEO (URL points directly to video) 
It works for those videos which are hosted/displayed into learning platforms.**(DOES NOT WORK IN ALL CASES)**

Actually you should be able **to copy the pointing-to-video-http-address exploring the web as developer in your browser (DEVELOPER TOOLS)**
After that just click on the play button and search for the video source URL within the sourceCode. 

This copied URL, must be given into the "Enter the Video URL:" cell

In [ ]:
import requests
import whisper
from IPython.display import clear_output

Task = "Translate to English" #@param ["Transcript to Language", "Translate to English", "Transcribe"]
task = "translate" if Task == "Translate to English" else "transcribe"

video_url = input('Enter the video URL: ')
response = requests.get(video_url)
open('record.wav', 'wb').write(response.content)

!whisper "record.wav" --task {task} --model medium --verbose True

clear_output()
if task == "translate":
    print("-- TRADUCCIÓN A INGLÉS --\n")
else:
    print("-- TRANSCRIPCIÓN --\n")


with open('record.txt', 'r') as f:
    text = f.read()
    print(text)


with open('transcription.txt', 'w') as f:
    f.write(text)


-- TRADUCCIÓN A INGLÉS --

Hello everyone, how are you? Welcome to this Sunday class, which conveniently falls
in the middle of a bridge and which I think justifies why right now I'm the only
assistant. I'm going to wait a couple of initial minutes until someone joins and in case
of not joining, okay, I see that people are joining, in case of not having more people, I will start.
We are in the wrong room. I'm in the wrong room. We are in the other one. But where are we?
I have the one from week one. Okay, I'm going there. Okay, okay, perfect.
I'm in the two. People have come to look for you.
Okay, okay, well, I'm going over there to see if ...



# 2. TRANSCRIBE FROM A VIDEO SAVED IN YOUR LOCAL COMPUTER

Just explore your folders through the widget "Search in your computer" (runs automaticatly)

⚠ Remember it shall takes more time because your video will be uploaded to GoogleColab Environment

💡 Notice that in:

 _!whisper "{video_path}" --task {task} --model medium --verbose True_ 

 verbose True ables to see the text out put in real time.
 
**You can desable it replacing True -> False**

In [ ]:
from google.colab import files
import whisper
from IPython.display import clear_output

Task = "Transcribe" #@param ["Transcript to Language", "Translate to English", "Transcribe"]
task = "translate" if Task == "Translate to English" else "transcribe"

uploaded = files.upload()

video_path = list(uploaded.keys())[0]

!whisper "{video_path}" --task {task} --model medium --verbose True

clear_output()
if task == "translate":
    print("-- TRADUCCIÓN A INGLÉS --\n")
else:
    print("-- TRANSCRIPCIÓN --\n")

# Obtener el nombre del archivo sin la extensión
file_name = video_path.split('.')[0]

with open(f'{file_name}.txt', 'r') as f:
    text = f.read()
    print(text)

with open('transcription.txt', 'w') as f:
    f.write(text)



-- TRANSCRIPCIÓN --

Hola a todos, ¿qué tal estáis? Bienvenidos a esta clase del domingo que convenientemente
cae en mitad de un puente y que creo que justifica el por qué ahora mismo solo estoy
yo de asistente. Voy a esperar un par de minutillos iniciales hasta que se sume alguien y en caso
de no sumarse, ya veo que se está uniendo gente, en caso de no tener más gente pues arrancaré yo.
Es que está en la sala equivocada. Estoy en la sala equivocada. Estamos en la otra y he dicho que raro está el profesor.
Pero ¿en cuál estamos? Yo tengo la de semana uno domingo.
Vale, pues voy para allá. Vale, vale, vale, perfecto.
Yo también, estoy en las dos. La gente ha venido a buscar.
Vale, vale, pues voy por allá a ver si encuentro.



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 3. GETTING TRANSCRIPTION FROM A YOUTUBE VIDEO

**3.1 Get a fast and simple transcription.**

💡 just copy-paste the video id.

 I.e.: https ://www.youtube.com/watch?v=8S0FDjFBj8o&t=115s

  video id = 8S0FDjFBj8o


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = '-MPLSfD6Dco'  # Replace VIDEO_ID with the actual video ID
transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

# Get the list of available transcriptions
transcript = transcript_list.find_transcript(['en', 'es'])

# If a transcription is found, get the language and the corresponding transcription
if transcript:
    language_code = transcript.language
    transcript_text = transcript.fetch()

    # Print the transcription
    for segment in transcript_text:
        text = segment['text']
        print(text)
else:
    print("No transcription was found for the video.")



Hola a todos Bienvenidos a Bueno pues
una nueva sesión de en esta semana
online de los casos de uso en
agricultura hoy vamos a hablar un
poquito sobre mercado sobre mercado la
situación del mercado Black Tech el
mercado de las inversiones En tecnología
en agricultura tendencias en el sector Y
también vamos a tener una pequeña charla
sobre Bueno pues el uso legal de los
datos enmarcado en el sector
agroalimentario también vale Bueno al
final
se dan una serie de
situaciones vale hemos sido repasando en
las primeras sesiones No pues esas esos
sistemas que son capaces de generar
distintas distintas tipos de datos
digitales que son susceptibles de
incorporar en esa copa analítica que
venimos comentando no y que nos pueden
ayudar en la toma de decisiones pero
todos estos sistemas que están presentes
a lo largo de toda la cadena
agroalimentaria
generan datos Y esos datos no sólo
le preocupan o le interesan a
el generador de esos datos por decirlo
una manera no todos esos datos de
estaciones m

**3.2 transcription and translation selectable with dropdown. There is a limit of translations per day**

After you choose any language translation, please be patient, it shall takes a while depending of the text length. 

Just wait until "process completed" is shown.
Once completed, you can take the transcript/translation .txt file from Colab folder


Here you can take a look to the languages codes:
https://cloud.google.com/translate/docs/languages

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from translate import Translator
import ipywidgets as widgets
from IPython.display import display

# Obtener el ID del video
_id = input("Ingrese el ID del video de YouTube: ")

# Crear el widget de selección de idioma
idioma_destino_widget = widgets.Dropdown(
    options=['es', 'en', 'fr', 'de', 'no_translation'],  # Agrega aquí los códigos de idioma que deseas ofrecer al usuario
    description='Idioma de destino:'
)

# Mostrar el widget de selección de idioma
display(idioma_destino_widget)

# Función para manejar el evento de selección de idioma
def seleccionar_idioma(change):
    global translator
    idioma_destino = change.new

    if idioma_destino == 'no_translation':
        # No realizar traducción
        translator = None
    else:
        translator = Translator(to_lang=idioma_destino)

    # Obtener la transcripción original
    transcript_list = YouTubeTranscriptApi.list_transcripts(_id)
    transcript = transcript_list.find_transcript(['en', 'es'])

    if transcript:
        language_code = transcript.language
        transcript_text = transcript.fetch()

        # Traducir o guardar la transcripción sin traducción según el idioma seleccionado
        with open('transcript.txt', 'w') as f:
            for segment in transcript_text:
                text = segment['text']
                if translator is None:
                    translation = text
                else:
                    translation = translator.translate(text)
                f.write(translation + '\n')

        print("Proceso completado.")
    else:
        print("No se encontró ninguna transcripción para el video.")

# Registrar la función de manejo de eventos
idioma_destino_widget.observe(seleccionar_idioma, names='value')


Ingrese el ID del video de YouTube: -MPLSfD6Dco


Dropdown(description='Idioma de destino:', options=('es', 'en', 'fr', 'de', 'no_translation'), value='es')

Proceso completado.


# SUMMARY

here you can obtain a text summary.

1. copy the .txt file name on the "sample_data" folder in Colab

2. paste it where showed in the code

3. Select how many word the summary should contain

In [ ]:


from summa import summarizer

with open('transcript.txt', 'r') as f:   # paste here your .txt file name
    text = f.read()

summary = summarizer.summarize(text, words=2000)  # write the number of words

with open('summary.txt', 'w') as f:
    f.write(summary)


#4. 🎙️ **Record and transcribe** [Choose the task]

herein you can record your own voice or conversation and transcribe it. Choose whether you want to transcribe from the original language or transcribe to english

In [ ]:
Task = "Transcript to Language" #@param ["Transcript to Language", "Translate to English"]
import numpy as np
import whisper
from scipy.io.wavfile import write
from IPython.display import clear_output

task = "translate" if Task == "Translate to English" else "transcribe" 

audio, sr = get_audio()
write('record.wav', sr, audio)

!whisper "record.wav" --task {task} --model medium --verbose False

clear_output()
if task == "translate":
  print("-- TRADUCCIÓN A INGLÉS --\n")
else:
  print("-- TRANSCRIPCIÓN A ESPAÑOL --\n")
  
print(open('record.wav.txt').read())

## 👉 Only **transcribe** audio to the original language. (X ► X)

In [ ]:
!whisper "/content/audio.mp3" --task transcribe --model medium --verbose False --output_dir audio_transcription

## 👉 Only **translate** audio from the original language to English. (X ► English)

In [ ]:
!whisper "/content/audio.mp3" --task translate --model medium